Hibridni sistem projektovan koriščenjem LightFm bibilioteke

Učitavanje neophodnih biblioteka

In [ ]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM 
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.evaluation import recall_at_k

Piprema seta--postoji isti set u LightFm biblioteci pa možemo korisiti njega

In [21]:
data = fetch_movielens(min_rating=4.0)

Proveravanje objekata matrice neophodne za model

In [11]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [22]:
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

definisanje metrika

In [23]:
def _get_metrics(model, train_set, test_set):
 
    train_set = train_set.tocsr()
    test_set = test_set.tocsr()
 
    train_set.data[train_set.data < 0] = 0.0
    test_set.data[test_set.data < 0] = 0.0
 
    train_set.eliminate_zeros()
    test_set.eliminate_zeros()
 
    return (precision_at_k(model, train_set).mean(),
            precision_at_k(model, test_set).mean(),
            auc_score(model, train_set).mean(),
            auc_score(model, test_set).mean(),
            recall_at_k(model, train_set).mean(),
            recall_at_k(model, train_set).mean())
            



funkcija za dobijanje liste preporuka

In [38]:
def get_recommendations(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
    
     
        #print out the results
        print("User with id number: %s" % user_id)
        print(" Known Movies that the user has liked:")

        for x in known_positives[:5]:
            print("        %s" % x)

        print("Recommended movies for the user:")

        for x in top_items[:10]:
            print("        %s" % x)


In [39]:
get_recommendations(model, data, [11, 22, 33, 44, 55])

User with id number: 11
 Known Movies that the user has liked:
        Get Shorty (1995)
        Mr. Holland's Opus (1995)
        Apollo 13 (1995)
        Star Wars (1977)
        Forrest Gump (1994)
Recommended movies for the user:
        Star Wars (1977)
        Schindler's List (1993)
        Silence of the Lambs, The (1991)
        Raiders of the Lost Ark (1981)
        Return of the Jedi (1983)
        Forrest Gump (1994)
        Braveheart (1995)
        Shawshank Redemption, The (1994)
        Fugitive, The (1993)
        Godfather, The (1972)
User with id number: 22
 Known Movies that the user has liked:
        Toy Story (1995)
        Twelve Monkeys (1995)
        Babe (1995)
        Mighty Aphrodite (1995)
        Postino, Il (1994)
Recommended movies for the user:
        Star Wars (1977)
        Fargo (1996)
        Monty Python and the Holy Grail (1974)
        Silence of the Lambs, The (1991)
        Pulp Fiction (1994)
        Princess Bride, The (1987)
        Raider

In [37]:
_get_metrics(model, data['train'], data['test'])

(0.5204883,
 0.08115632,
 0.9566875,
 0.91840386,
 0.18834887060274244,
 0.18834887060274244)